In [454]:
import requests
import json
import pandas as pd
import numpy as np
import sqlite3

# Load

In [455]:
# loop through the pages in order to gather all the entries in a single list
def get_results(data, key):
    result = []
    current_page = data
    has_next = True
    
    while(has_next):
        for data_point in current_page['results']:
            result.append(data_point)

        if current_page['next'] == None:
            has_next = False
        else:
            response = requests.get(current_page['next'])
            current_page = response.json()

    return result

In [456]:
# get the json from the API
response = requests.get("https://swapi.dev/api/?format=json")
root = response.json()

data = {}

# get json from the major columns (people, planets, etc.)
for key in root:
    response = requests.get(root[key] + '/?format=json')
    data[key] = get_results(response.json(), key)

# Transform

In [457]:
# write finished json to dataframes
df_people_extracted = pd.json_normalize(data['people'])
df_films_extracted = pd.json_normalize(data['films'])
df_vehicles_extracted = pd.json_normalize(data['vehicles'])

In [458]:
print('people')
display(df_people_extracted.tail(1))
print('films')
display(df_films_extracted.tail(1))
print('vehicles')
display(df_vehicles_extracted.tail(1))

people


,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
81,Tion Medon,206,80,none,grey,black,unknown,male,https://swapi.dev/api/planets/12/,[https://swapi.dev/api/films/6/],[https://swapi.dev/api/species/37/],[],[],2014-12-20T20:35:04.260000Z,2014-12-20T21:17:50.498000Z,https://swapi.dev/api/people/83/


films


,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url
5,Revenge of the Sith,3,War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/2/, https://s...","[https://swapi.dev/api/vehicles/33/, https://s...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-20T18:49:38.403000Z,2014-12-20T20:47:52.073000Z,https://swapi.dev/api/films/6/


vehicles


,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilots,films,created,edited,url
38,AT-RT,All Terrain Recon Transport,Kuat Drive Yards,40000,3.2,90,1,0,20,1 day,walker,[],[https://swapi.dev/api/films/6/],2014-12-20T20:47:49.189000Z,2014-12-20T21:30:21.772000Z,https://swapi.dev/api/vehicles/76/


#### People

In [459]:
df_people_cleaned = df_people_extracted.drop(columns=['homeworld', 'species', 'starships', 'created', 'edited'])

In [460]:
df_people_cleaned.replace(to_replace=[None, 'none', 'n/a', 'unknown'], value=np.nan, inplace=True)

In [461]:
df_people_cleaned['birth_year'] = df_people_cleaned['birth_year'].str.replace('BBY', '', regex=True)

In [462]:
df_people_cleaned['people_id'] = df_people_cleaned['url'].apply(
    lambda x: x.split('/')[5] if len(x) > 0 else np.nan
)
df_people_cleaned.drop(columns=['url'], inplace=True)

In [463]:
df_characters_exploded = df_people_cleaned.explode('films')
df_characters_exploded['films'] = df_characters_exploded['films'].apply(
    lambda x: x.split('/')[5] if len(x) > 0 else np.nan
)

df_characters = df_characters_exploded[[
    'people_id',
    'films',
]].rename(columns={'films': "film_id"})

In [464]:
df_characters

,people_id,film_id
0,1,1
0,1,2
0,1,3
0,1,6
1,2,1
...,...,...
79,81,1
79,81,6
80,82,5
80,82,6


In [465]:
df_vehicles_exploded = df_people_cleaned.explode('vehicles')
df_vehicles_exploded['vehicles'] = df_vehicles_exploded['vehicles'].apply(
    lambda x: x.split('/')[5] if type(x) != float else np.nan
)

df_vehicles_per_person = df_vehicles_exploded[[
    'people_id',
    'vehicles',
]].rename(columns={'vehicles': "vehicle_id"})

In [466]:
df_vehicles_per_person.dropna(inplace=True)

In [467]:
df_vehicles_per_person

,people_id,vehicle_id
0,1,14
0,1,30
4,5,30
9,10,38
10,11,44
10,11,46
12,13,19
16,18,14
30,32,38
42,44,42


In [468]:
df_people_cleaned.drop(columns=['films', 'vehicles'], inplace=True)

In [469]:
df_people_cleaned.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,people_id
0,Luke Skywalker,172,77,blond,fair,blue,19,male,1
1,C-3PO,167,75,NaN,gold,yellow,112,NaN,2
2,R2-D2,96,32,NaN,"white, blue",red,33,NaN,3
3,Darth Vader,202,136,NaN,white,yellow,41.9,male,4
4,Leia Organa,150,49,brown,light,brown,19,female,5


#### Films

In [470]:
df_films_cleaned = df_films_extracted.drop(columns=['planets','starships','species','created', 'edited'])

In [471]:
df_films_cleaned.replace(to_replace=[None, 'none', 'n/a', 'unknown'], value=np.nan, inplace=True)

In [472]:
df_films_cleaned['film_id'] = df_films_cleaned['url'].apply(
    lambda x: x.split('/')[5] if len(x) > 0 else np.nan
)
df_films_cleaned.drop(columns=['url'], inplace=True)

In [473]:
df_vehicles_exploded = df_films_cleaned.explode('vehicles')
df_vehicles_exploded['vehicles'] = df_vehicles_exploded['vehicles'].apply(
    lambda x: x.split('/')[5] if type(x) != float else np.nan
)

df_vehicles_per_film = df_vehicles_exploded[[
    'film_id',
    'vehicles',
]].rename(columns={'vehicles': "vehicle_id"})

In [474]:
df_vehicles_per_film.head()

,film_id,vehicle_id
0,1,4
0,1,6
0,1,7
0,1,8
1,2,8


In [475]:
df_films_cleaned.drop(columns=['characters', 'vehicles'], inplace=True)

In [476]:
df_films_cleaned.tail()

,title,episode_id,opening_crawl,director,producer,release_date,film_id
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,2
2,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,3
3,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,4
4,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,5
5,Revenge of the Sith,3,War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,6


#### Vehicles

In [477]:
df_vehicles_cleaned = df_vehicles_extracted.drop(columns=['created', 'edited', 'pilots', 'films'])

In [478]:
df_vehicles_cleaned.replace(to_replace=[None, 'none', 'n/a', 'unknown'], value=np.nan, inplace=True)

In [479]:
df_vehicles_cleaned['vehicle_id'] = df_vehicles_cleaned['url'].apply(
    lambda x: x.split('/')[5] if len(x) > 0 else np.nan
)
df_vehicles_cleaned.drop(columns=['url'], inplace=True)

In [480]:
df_vehicles_cleaned.head()

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,vehicle_id
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,4
1,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,6
2,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,NaN,repulsorcraft,7
3,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,NaN,6.4,1200,1,0,65,2 days,starfighter,8
4,Snowspeeder,t-47 airspeeder,Incom corporation,NaN,4.5,650,2,0,10,NaN,airspeeder,14


# Load

In [481]:
conn = sqlite3.connect('starwars_database.db')
c = conn.cursor()

c.execute(f"CREATE TABLE IF NOT EXISTS people ({', '.join(df_people_cleaned.columns)})")
conn.commit()
df_people_cleaned.to_sql('people', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS films ({', '.join(df_films_cleaned.columns)})")
conn.commit()
df_films_cleaned.to_sql('films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles ({', '.join(df_vehicles_cleaned.columns)})")
conn.commit()
df_vehicles_cleaned.to_sql('vehicles', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS characters ({', '.join(df_characters.columns)})")
conn.commit()
df_characters.to_sql('characters', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles_per_person ({', '.join(df_vehicles_per_person.columns)})")
conn.commit()
df_vehicles_per_person.to_sql('vehicles_per_person', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles_per_film ({', '.join(df_vehicles_per_film.columns)})")
conn.commit()
df_vehicles_per_film.to_sql('vehicles_per_film', conn, if_exists='replace', index = False)

49

# Analysis

In [482]:
con = sqlite3.connect('starwars_database.db')

#### INNER

Find the number of films that the respective characters appear in in order to show how central they are to the film series.

In [483]:
pd.read_sql(
    '''
        SELECT 
            p.name,
            count(*) AS nb_films
        FROM 
            people p
        INNER JOIN 
            characters c ON p.people_id = c.people_id
        GROUP BY
            p.name
        ORDER BY 
            nb_films DESC 
        LIMIT 10
    ''', 
    con
)

,name,nb_films
0,R2-D2,6
1,Obi-Wan Kenobi,6
2,C-3PO,6
3,Yoda,5
4,Palpatine,5
5,Luke Skywalker,4
6,Leia Organa,4
7,Darth Vader,4
8,Chewbacca,4
9,Wedge Antilles,3


#### LEFT

Show which vehicle class can be conducted by the greatest majority of people.

In [484]:
pd.read_sql(
    '''
        SELECT 
            v.vehicle_class,
            count(*) AS nb_people
        FROM 
            vehicles v
        INNER JOIN 
            vehicles_per_person vpp ON v.vehicle_id = vpp.vehicle_id
        GROUP BY
            v.vehicle_class
        ORDER BY 
            nb_people DESC 
        LIMIT 10
    ''', 
    con
)

,vehicle_class,nb_people
0,speeder,4
1,airspeeder,4
2,submarine,2
3,wheeled walker,1
4,walker,1
5,repulsorcraft,1


#### GROUP BY

Show the distribution of gender of people.

In [485]:
pd.read_sql(
    '''
        SELECT 
            p.gender,
            count(*) AS nb_people
        FROM 
            people p
        WHERE
            p.gender IS NOT NULL
        GROUP BY
            p.gender
    ''', 
    con
)

,gender,nb_people
0,female,17
1,hermaphrodite,1
2,male,60


#### HAVING

Find out which films have more than 30 characters.

In [486]:
pd.read_sql(
    '''
        SELECT 
            f.title,
            count(*) AS nb_characters
        FROM 
            films f
        JOIN 
            characters c ON f.film_id = c.film_id
        GROUP BY
            f.title
        HAVING
            nb_characters > 30
        LIMIT 10
    ''', 
    con
)

,title,nb_characters
0,Attack of the Clones,40
1,Revenge of the Sith,34
2,The Phantom Menace,34


#### CTE

Show the number of female vs male pilots/drivers of vehicles throughout the movies

In [487]:
pd.read_sql(
    '''
        WITH female_drivers AS (
            SELECT 
                p.gender,
                COUNT(*) AS nb_drivers
            FROM 
                people p 
                INNER JOIN vehicles_per_person vpp ON vpp.people_id = p.people_id
                WHERE p.gender = 'female'
        ),
                    
        male_drivers AS (
            SELECT 
                p.gender,
                COUNT(*) AS nb_drivers
            FROM 
                people p 
                INNER JOIN vehicles_per_person vpp ON vpp.people_id = p.people_id
                WHERE p.gender = 'male'
        )
        
        SELECT 
            *
        FROM
            female_drivers 
        
        UNION
        
        SELECT 
            *
        FROM 
            male_drivers
    ''', 
    con
)

,gender,nb_drivers
0,female,2
1,male,11
